In [1]:
# Imports
import pandas as pd
import seaborn as sns
import historicdutchweather
import pytz
from typing import Tuple
import pvlib
from datetime import datetime
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Business Understanding
We kijken naar zonnepaneeldata over een periode van net iets minder naar twee jaar. De metingen bestaan uit het wattage dat elke minuut door de panelen werd opgewekt. Hierbij is de data al omgezet naar watt per vierkante meter.

We zijn op zoek naar de efficientie van de panelen. Deze kan berekend worden door de theoretische hoeveelheid opbrengst te nemen en deze te verlagen met een bepaalde factor om de werkelijke meting te benaderen. Deze factor is de efficientie.

Om deze berekening te moeten doen, maken we gebruik van de pvlib library. Hierbij is het nodig om de helling van het zonnepaneel met het dak te weten (de tilt). En ook de richting van het paneel (de azimuth). De azimuth loopt van 0 graden noord, naar 90 graden oost, naar 180 graden zuid en dan naar 270 graden west.

# Data Understanding
## Stroomproductiedata ophalen

In [16]:
df = pd.read_csv('../../bijlagen/dataset.csv')
df['time'] = pd.to_datetime(df['time'])

In [17]:
# Bekijk de eerste paar rijen van de DataFrame
print(df.head())

# Verkrijg statistische samenvattingen van de DataFrame
print(df.describe())

# Verkrijg informatie over de DataFrame, zoals kolomnamen en datatypes
print(df.info())

# Controleer op ontbrekende waarden in de DataFrame
print(df.isnull().sum())

                       time  production
0 2020-04-10 10:31:35+00:00   96.666667
1 2020-04-10 10:32:35+00:00   96.666667
2 2020-04-10 10:33:35+00:00   93.333333
3 2020-04-10 10:34:35+00:00  100.000000
4 2020-04-10 10:35:36+00:00  103.333333
         production
count  1.266008e+06
mean   2.181287e+01
std    7.722842e+01
min    0.000000e+00
25%    0.000000e+00
50%    0.000000e+00
75%    2.000000e+01
max    2.206667e+03
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1266008 entries, 0 to 1266007
Data columns (total 2 columns):
 #   Column      Non-Null Count    Dtype              
---  ------      --------------    -----              
 0   time        1266008 non-null  datetime64[ns, UTC]
 1   production  1266008 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 19.3 MB
None
time          0
production    0
dtype: int64


## Weerdataset ophalen

In [18]:
# Laad de weersdata in een DataFrame
df_weather = pd.read_csv('../../bijlagen/weather.csv')

# Zet de 'Unnamed: 0' kolom om naar datetime formaat
df_weather['Unnamed: 0'] = pd.to_datetime(df_weather['Unnamed: 0'])

# Hernoem de 'Unnamed: 0' kolom naar 'time'
df_weather.rename(columns={'Unnamed: 0': 'time'}, inplace=True)

# Bekijk de eerste paar rijen van de weerdata DataFrame
df_weather.head()

,time,T,FH,DD,Q,DR,RH,U,N
0,2020-04-10 00:00:00+00:00,4.461015,2.604330,56.449253,-1.482197e-323,-1.482197e-323,-1.482197e-323,85.211640,1.947349
1,2020-04-10 01:00:00+00:00,4.014926,2.604330,58.753726,-1.482197e-323,-1.482197e-323,-1.482197e-323,89.105820,7.088386
2,2020-04-10 02:00:00+00:00,3.437970,2.249255,62.304473,-1.482197e-323,-1.482197e-323,-1.482197e-323,91.230447,8.000000
3,2020-04-10 03:00:00+00:00,3.019567,2.124627,54.608945,-1.482197e-323,-1.482197e-323,-1.482197e-323,92.124627,8.000000
4,2020-04-10 04:00:00+00:00,3.175657,2.355075,60.000000,6.449253e-01,-1.482197e-323,-1.482197e-323,92.355075,8.000000


**Omzetten naar Datetime:**
- df_weather['Unnamed: 0'] = pd.to_datetime(df_weather['Unnamed: 0']): Zet de kolom 'Unnamed: 0', die vermoedelijk tijdstempels bevat, om naar een datetime formaat. Dit is belangrijk voor tijdsgebonden analyses en het correct verwerken van tijdsdata.

**Hernoemen van Kolom:**
- df_weather.rename(columns={'Unnamed: 0': 'time'}, inplace=True): Hernoemt de kolom 'Unnamed: 0' naar 'time'. Dit maakt de kolomnaam duidelijker en handiger voor verdere verwerking en analyses.

## Data Preparation

## Resampling naar xx minuten

In [19]:
# Zet de 'time' kolom als de index van de DataFrame
df.set_index('time', inplace=True)

# Resample de data naar 10 minuten intervallen en bereken het gemiddelde
df_resampled = df.resample('10T').mean()  # '10T' staat voor 10 minuten

df_resampled.head()

C:\Users\MSI\AppData\Local\Temp\ipykernel_20932\3170985748.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df.resample('10T').mean()  # '10T' staat voor 10 minuten


,production
time,
2020-04-10 10:30:00+00:00,178.148148
2020-04-10 10:40:00+00:00,80.333333
2020-04-10 10:50:00+00:00,86.666667
2020-04-10 11:00:00+00:00,172.000000
2020-04-10 11:10:00+00:00,166.333333


In [20]:
# Zet de 'time' kolom als de index van de weerdata DataFrame
df_weather.set_index('time', inplace=True)

# Resample de weerdata naar 10 minuten intervallen en bereken het gemiddelde
weather_resampled = df_weather.resample('10T').mean()  # '10T' staat voor 10 minuten

C:\Users\MSI\AppData\Local\Temp\ipykernel_20932\1790055929.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  weather_resampled = df_weather.resample('10T').mean()  # Zelfde frequentie als zonnepaneeldata


## Weer en zonnepanelen combineren

In [21]:
# Combineer de geresamplede zonnepaneeldata en weerdata op basis van tijd
combined_df = df_resampled.join(df_weather, how='inner')

# Interpoleer de ontbrekende waarden in de gecombineerde DataFrame
combined_df.interpolate(method='linear', inplace=True)

## Filter op zonnige dagen

In [34]:
import pandas as pd

# Stel een drempelwaarde in voor zonne-instraling
threshold = 200  # bijv. 200 W/m²

# Groepeer op dagbasis en bereken de gemiddelde zonne-instraling per dag
daily_mean_solar_radiation = combined_df['Q'].resample('D').mean()

# Filter de dagen met zonne-instraling boven de drempelwaarde
sunny_days = daily_mean_solar_radiation[daily_mean_solar_radiation > threshold]

# Verkrijg de datums van zonnige dagen
sunny_days_dates = sunny_days.index.date

# Filter de data om alleen de zonnige dagen te bevatten
sunny_days_data = combined_df[combined_df.index.date[:, None] == sunny_days_dates]

# Bekijk de gefilterde data
sunny_days_data.head()

,production,T,FH,DD,Q,DR,RH,U,N
time,,,,,,,,,
2020-04-10 11:00:00+00:00,172.000000,13.783896,3.355075,58.159692,243.164942,-1.482197e-323,-1.482197e-323,38.211640,7.316289
2020-04-10 12:00:00+00:00,83.000000,15.003179,3.124627,52.956713,247.188076,-1.482197e-323,-1.482197e-323,38.644925,7.544193
2020-04-10 13:00:00+00:00,167.333333,15.288538,2.479702,62.898506,224.543431,-1.482197e-323,-1.482197e-323,36.040596,7.052651
2020-04-10 14:00:00+00:00,87.000000,15.613761,3.124627,54.797021,199.782821,-1.482197e-323,-1.482197e-323,35.000000,7.684217
2020-04-10 15:00:00+00:00,54.000000,15.310284,3.355075,45.855220,132.824630,-1.482197e-323,-1.482197e-323,37.991199,6.105303


## Uitleg van de Kolommen in de DataFrame

- **time:** De tijdstempel waarop de gegevens zijn gemeten.
Formaat: Datum en tijd in ISO 8601-formaat (bijvoorbeeld 2020-04-10 11:00:00+00:00).
- **production:** De hoeveelheid elektriciteit geproduceerd door de zonnepanelen in watt per vierkante meter (W/m²) op dat specifieke tijdstip.
- **T:** De temperatuur op het moment van de meting, waarschijnlijk in graden Celsius.
- **FH:** De hoeveelheid directe zonnestraling op de horizontale vlakte, mogelijk in watt per vierkante meter (W/m²).
- **DD:** De windrichting, vaak in graden (0° = Noord, 90° = Oost, etc.).
- **Q:** De globale zonne-instraling op de panelen, meestal in watt per vierkante meter (W/m²). Dit is de kolom die gebruikt wordt om zonnige dagen te identificeren.
- **DR:** Deze kolom bevat waarschijnlijk ontbrekende of foutieve waarden, aangeduid met zeer kleine getallen dicht bij nul (-1.482197e-323). Dit kan wijzen op een probleem met de data of een placeholder voor ontbrekende gegevens.
- **RH:** De relatieve luchtvochtigheid op het meetmoment, vaak uitgedrukt als een percentage.
- **U:** De windsnelheid, mogelijk in meters per seconde (m/s).
- **N:** De hoeveelheid bewolking of de zonnigheid, mogelijk in een schaal van 0 tot 10 of een andere eenheid.

# Modeling

In [31]:
# Gegeven:
def model_zonnestraling(tilt:float, azimuth:float, tijdsreeks:pd.core.indexes.datetimes.DatetimeIndex, lat:float, lon:float) -> pd.DataFrame:
    """Berekent de hoeveelheid stroom die een paneel maximaal produceert met 100% efficientie"""

    zonne_positie_aan_hemel = pvlib.solarposition.get_solarposition(tijdsreeks, lat, lon)

    #locatie = pvlib.location.Location(latitude=lat, longitude=lon, name='Plaatsnaam', tz=pytz.timezone(r'Europe/Amsterdam'))
    locatie = pvlib.location.Location(latitude=lat, longitude=lon, name='Plaatsnaam', tz=pytz.timezone(r'UTC'))
    heldere_hemel = locatie.get_clearsky(tijdsreeks)

    instraling = pvlib.irradiance.get_total_irradiance(tilt,
                                                       azimuth,
                                                       zonne_positie_aan_hemel['zenith'],
                                                       zonne_positie_aan_hemel['azimuth'],
                                                       heldere_hemel.dni,
                                                       heldere_hemel.ghi,
                                                       heldere_hemel.dhi)

    return instraling['poa_global'].to_frame().rename(columns={'poa_global':'zonlicht'})

In [37]:
def model_paneel(tilt: float, azimuth: float, efficientie: float, tijdsreeks: pd.DatetimeIndex, lat: float, lon: float) -> pd.DataFrame:
    """Berekent de hoeveelheid geabsorbeerd zonlicht op een paneel op basis van de efficiëntie"""

    # Gebruik model_zonnestraling om de zonne-instraling te berekenen met 100% efficiëntie
    df_instraling = model_zonnestraling(tilt, azimuth, tijdsreeks, lat, lon)

    # Pas de efficiëntie toe om de geabsorbeerde zonne-energie te berekenen
    df_model = df_instraling.copy()
    df_model['geabsorbeerd'] = df_model['zonlicht'] * efficientie

    return df_model

In [38]:
def loss(params: Tuple[float, float, float], df_waarneming: pd.DataFrame, lat: float, lon: float) -> float:
    """Bereken de verliesfunctie voor de optimalisatie"""

    tilt, azimuth, efficientie = params
    tijdsreeks = df_waarneming.index

    # Verkrijg modelresultaten
    df_model = model_paneel(tilt, azimuth, efficientie, tijdsreeks, lat, lon)

    # Bereken de kwadratische afwijking tussen waarnemingen en model
    verschil = df_waarneming['production'] - df_model['geabsorbeerd']
    verlies = (verschil ** 2).mean()  # Gemiddelde kwadratische afwijking

    return verlies

In [48]:
# Definieer de locatiecoördinaten
locations = {
    'Amsterdam': {'lat': 52.379189, 'lon': 4.899431},
    'Cairo': {'lat': 30.0444, 'lon': 31.2357},
    'Sydney': {'lat': -33.8688, 'lon': 151.2093}
}

# Definieer initiële waarden voor tilt, azimuth en efficiëntie
tilt_init = 35
azimuth_init = 180
efficientie_init = 0.2

# Functie om optimalisatie uit te voeren
def optimize_location(lat: float, lon: float, df: pd.DataFrame) -> pd.Series:
    result = minimize(
        fun=loss,
        x0=(tilt_init, azimuth_init, efficientie_init),
        args=(df, lat, lon),
        method='L-BFGS-B',
        bounds=[(0, 90), (0, 360), (0, 1)]
    )
    return pd.Series(result.x, index=['Tilt', 'Azimuth', 'Efficientie'])

# Resultaten voor elke locatie
results = {}
for city, coords in locations.items():
    result = optimize_location(coords['lat'], coords['lon'], combined_df)
    results[city] = result

# Print de resultaten
print("\nVergelijking van Optimalisatie Resultaten:")
for city, result in results.items():
    print(f"{city}:")
    print(f"  Tilt:        {result['Tilt']:.1f}°")
    print(f"  Azimuth:     {result['Azimuth']:.1f}°")
    print(f"  Efficientie: {result['Efficientie']*100:.1f}%")


Vergelijking van Optimalisatie Resultaten:
Amsterdam:
  Tilt:        0.8°
  Azimuth:     182.4°
  Efficientie: 9.2%
Cairo:
  Tilt:        0.0°
  Azimuth:     152.8°
  Efficientie: 6.7%
Sydney:
  Tilt:        90.0°
  Azimuth:     180.1°
  Efficientie: 5.5%


## Evaluation
Op basis van de optimalisatie van zonnepanelenparameters voor de drie locaties kunnen we de volgende conclusies trekken:

- **Amsterdam:** Lage tilt (0.8°) en azimuth dicht bij het zuiden (182.4°) wijzen op een optimalisatie voor relatief gematigde zonne-instraling. De hogere efficiëntie (9.2%) kan worden toegeschreven aan het relatief constante zonlicht gedurende het jaar.

- **Cairo:** Geen tilt (0.0°) en een azimuth van 152.8° wijzen op een focus op het minimaliseren van schaduw, passend bij de zeer zonnige omstandigheden. Lagere efficiëntie (6.7%) kan te maken hebben met intensieve zoninstraling die moeilijk te benutten is zonder een optimale tilt.

- **Sydney:** Maximale tilt (90.0°) en een azimuth dicht bij het zuiden (180.1°) suggereren dat zonnepanelen zijn afgestemd op een constante hoge zonnestraling. De lagere efficiëntie (5.5%) kan het gevolg zijn van een mix van hoge zonnestraling en schaduwwerking, met mogelijk minder constante zoninstraling gedurende het jaar.